In [1]:
import numpy as np
import slam
from importlib import reload

model, matching = slam.get_models()

xFormers not available
xFormers not available


Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)


In [3]:
image_prefix = "indoor1"

In [4]:
images = slam.get_images(image_prefix)
match_inputs = [slam.preprocess_image_matcher(x) for x in images]
depth_inputs = [slam.preprocess_image_depth(x) for x in images]
depths = [slam.get_depth(image=x, model=model) for x in depth_inputs]
matches = slam.get_matches(inputs=match_inputs, model=matching, num_matches=-1)
matches_3d = slam.project_matches_to_3d(matches, depths)

In [5]:
transforms = [slam.get_transform(m[0], m[1]) for m in matches_3d]
angles = [slam.decompose_rotation_matrix(t[0]) for t in transforms]
elevation = [a[2] for a in angles]
azimuth = [a[1] for a in angles]

for a, t in zip(angles, transforms):
    x,y,z = a
    print("{: <15}".format(f"x deg {x:.3f}") + "{: <15}".format(f"y deg {y:.3f}") + "{: <15}".format(f"z deg {z:.3f}"))

    x,y,z = t[1]
    print("{: <15}".format(f"x {x:.3f}") + "{: <15}".format(f"y {y:.3f}") + "{: <15}".format(f"z {z:.3f}"))
    print()

x deg 0.170    y deg 9.402    z deg -3.684   
x -0.044       y 0.007        z 0.177        

x deg 4.186    y deg 19.992   z deg 0.488    
x 0.314        y -0.153       z -0.353       

x deg 2.990    y deg 18.196   z deg -1.953   
x 0.200        y -0.186       z -0.052       

x deg -0.750   y deg 13.131   z deg -1.955   
x 0.020        y 0.029        z -0.175       

x deg -1.683   y deg 7.781    z deg -6.154   
x -0.195       y 0.128        z -0.286       



In [23]:
# idx = 1
# fig = slam.plot_matches(idx, images, matches)

In [7]:
# fig = create_pcd_fig(idx, depths, images)
# fig.show()

In [8]:
data = slam.training_preprocessor(
    image_prefix=image_prefix, 
    model=model,
    matching=matching,
    force_reload=False)


In [9]:
reload(slam)
df = slam.pd.DataFrame(data, columns=['x', 'y', 'z', 'xd', 'yd', 'zd', 'r', 'g', 'b'])
df['pos'] = list(zip(df['x'], df['y'], df['z']))
df['dir'] = list(zip(df['xd'], df['yd'], df['zd']))
df['rgb'] = list(zip(df['r'], df['g'], df['b']))
df = df[['pos', 'dir', 'rgb']]
df.head()

,pos,dir,rgb
0,"(0.0, 0.0, 0.0)","(-0.35689106583595276, -0.35689106583595276, 0...","(0.4627451002597809, 0.3843137323856354, 0.270..."
1,"(0.0, 0.0, 0.0)","(-0.3556877374649048, -0.35706639289855957, 0....","(0.48627451062202454, 0.40392157435417175, 0.2..."
2,"(0.0, 0.0, 0.0)","(-0.35448265075683594, -0.35724127292633057, 0...","(0.47843137383461, 0.3960784375667572, 0.28235..."
3,"(0.0, 0.0, 0.0)","(-0.35327577590942383, -0.35741570591926575, 0...","(0.4588235318660736, 0.3764705955982208, 0.262..."
4,"(0.0, 0.0, 0.0)","(-0.35206714272499084, -0.3575897514820099, 0....","(0.4588235318660736, 0.3764705955982208, 0.262..."


In [10]:
df.isna().sum()

pos    0
dir    0
rgb    0
dtype: int64

In [22]:
fig = slam.go.Figure()
positions = df['pos'].unique()
colors = list(slam.mcolors.CSS4_COLORS.values())[65:][:len(positions)]
for i, pos in enumerate(positions):
    for dir in df[df['pos'] == pos]['dir'][::2000]:
        try:
            trace = slam.go.Scatter3d(
                x=[pos[0], pos[0] + dir[0]],
                y=[pos[1], pos[1] + dir[1]],
                z=[pos[2], pos[2] + dir[2]],
                mode='lines',
                line=dict(color=colors[i], width=1)
            )

            fig.add_trace(trace)
        except:
            print(pos, dir)
fig.update_layout(
    scene=dict(
        aspectmode='data',
        bgcolor='rgba(0,0,0,0)',
        xaxis=dict(gridcolor='white', showbackground=False, color='white'),
        yaxis=dict(gridcolor='white', showbackground=False, color='white'),
        zaxis=dict(gridcolor='white', showbackground=False, color='white')
    ),
    showlegend=False,
    paper_bgcolor='rgba(0,0,0,0)'
)
fig.update_layout(
    width=800,
    height=800
)

fig.show()